# Detector e leitor de placas de carro

### Manuela Castilla e Thiago Verardo

### Objetivo:

Este projeto tem como objetivo a criação de um modelo capaz de identificar e ler uma placa de carro através de um vídeo.

In [1]:
import numpy as np
from math import atan2, cos, sin, sqrt, pi
import cv2
import imutils
import pytesseract
from skimage.segmentation import clear_border

# Passar o path para SEU pytesseract 
 pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'

Criando as funções de reconhecimento da orientação do veículo (3d)

In [2]:
def draw_axis(img, p_, q_, colour, scale):
    p = list(p_)
    q = list(q_)
    angle = atan2(p[1] - q[1], p[0] - q[0]) # radianos
    hypotenuse = sqrt((p[1] - q[1]) * (p[1] - q[1]) + (p[0] - q[0]) * (p[0] - q[0]))
    # realizamos a escala das setas
    q[0] = p[0] - scale * hypotenuse * cos(angle)
    q[1] = p[1] - scale * hypotenuse * sin(angle)
    cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), colour, 1, cv2.LINE_AA)
    
    # novas setas
    p[0] = q[0] + 9 * cos(angle + pi / 4)
    p[1] = q[1] + 9 * sin(angle + pi / 4)
    cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), colour, 1, cv2.LINE_AA)
    p[0] = q[0] + 9 * cos(angle - pi / 4)
    p[1] = q[1] + 9 * sin(angle - pi / 4)
    cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), colour, 1, cv2.LINE_AA)
    
def get_orientation(pts, img):
    sz = len(pts)
    data_pts = np.empty((sz, 2), dtype=np.float64)
    for i in range(data_pts.shape[0]):
        data_pts[i,0] = pts[i,0,0]
        data_pts[i,1] = pts[i,0,1]
        
    # analise PCA
    mean = np.empty((0))
    mean, eigenvectors, eigenvalues = cv2.PCACompute2(data_pts, mean)
    
    # guarda o centro do objeto analisado
    cntr = (int(mean[0,0]), int(mean[0,1]))
    cv2.circle(img, cntr, 3, (255, 0, 255), 2)
    p1 = (cntr[0] + 0.02 * eigenvectors[0,0] * eigenvalues[0,0], cntr[1] + 0.02 *  eigenvectors[0,1] * eigenvalues[0,0])
    p2 = (cntr[0] - 0.02 * eigenvectors[1,0] * eigenvalues[1,0], cntr[1] - 0.02 * eigenvectors[1,1] * eigenvalues[1,0])
    draw_axis(img, cntr, p1, (0, 150, 0), 1)
    draw_axis(img, cntr, p2, (200, 150, 0), 5)
    angle = atan2(eigenvectors[0,1], eigenvectors[0,0]) # orientation in radians
    return angle

Transformando o video (video.MOV) em frames para poderem ser analisados.

In [3]:
import time
import math

cap = cv2.VideoCapture('video.MOV')

frameRate = cap.get(5) #frame rate

x=1
while(cap.isOpened()):
    frameId = cap.get(1) #frame atual
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename = './videoPics/image' +  str(int(x)) + ".jpg";x+=1
        cv2.imwrite(filename, frame)
cap.release()

Criando a funções para a detecção da placa.

In [5]:
def tratamento_imagem_deteccao_ret(img):
    grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # determinar as partes mais escuras com background claro
    rectKern = cv2.getStructuringElement(cv2.MORPH_RECT, (13, 5))
    blackhat = cv2.morphologyEx(grey, cv2.MORPH_BLACKHAT, rectKern)

    # fecha onde ha poucas partes escuras para encontrar a parte clara maior
    squareKern = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    light = cv2.morphologyEx(grey, cv2.MORPH_CLOSE, squareKern)
    light = cv2.threshold(light, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    # Calculo do gradX a partir do blackhat
    gradX = cv2.Sobel(blackhat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
    gradX = np.absolute(gradX)
    (minVal, maxVal) = (np.min(gradX), np.max(gradX))
    gradX = 255 * ((gradX - minVal) / (maxVal - minVal))
    gradX = gradX.astype("uint8")
    
    # Aplicando blurr, fechamento da imagem e threshold para
    # obtermos a representação do gradiente (metodo Otsu)
    gradX = cv2.GaussianBlur(gradX, (5, 5), 0)
    gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKern)
    thresh = cv2.threshold(gradX, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    
    # Limpando a imagem com erosão e dilatação
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)
    
    # realizando um AND bit a bit com a parte clara da imagem
    thresh = cv2.bitwise_and(thresh, thresh, mask=light)
    thresh = cv2.dilate(thresh, None, iterations=2)
    thresh = cv2.erode(thresh, None, iterations=1)

    # Encontrando os contornos e devolvendo sortidos e apenas os maiores (prováveis placas)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:5]
    
    
    return cnts, grey

In [6]:
def license_detection(cnts, grey):
    # inicialização de variáveis
    lpCnt = None
    roi = None
    minAR = 4
    maxAR = 6
    clearBorder = False
    # percorrendo os candidatos para placa
    ar_list = []
    lp = None
    for c in cnts:
        # Pegando a proporção do candidato
        (x, y, w, h) = cv2.boundingRect(c)
        ar = w / float(h)
        if ar >= minAR and ar <= maxAR:
        # Guardando o contorno da placa
            lpCnt = c
            lp = grey[y:y + h, x:x + w]
            roi = cv2.threshold(lp, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
            # Checando se ainda precisa limpar algum ruído
            if clearBorder:
                roi = clear_border(roi)
            # Pegando a orientação do veículo a partir da função criada anteriormente
            get_orientation(c,img)
            break
    
    return roi, lp, x, y


Criando as variáveis para o tesseract.

In [7]:
# Passando os parametros para o Tesseract
alphanumeric = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
options = "-c tessedit_char_whitelist={}".format(alphanumeric)
options += " --psm {}".format(7)

Rodando as funções com as imagens capturadas do vídeo desejado.

In [8]:
lp_test_list = []
conta = 0
for i in range(29):
    
    # Percorrendo todas as imagens do vídeo para identificar placas e orientação
    img = cv2.imread(f"videoPics/image{i+1}.jpg")
    cnts, grey = tratamento_imagem_deteccao_ret(img)
    roi, licensePlate, x, y = license_detection(cnts, grey)
    
    # Inicializando o texto
    lpText = None
    if roi is not None:
        lpText = pytesseract.image_to_string(licensePlate, config=options)
        if lpText not in lp_test_list:
            lp_test_list.append(lpText)
            newstr = lpText.rstrip()
            
            # Colocando o texto identificado na imagem final
            cv2.putText(img, str(lpText[:-2]), (x + 20, y - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 0), lineType=cv2.LINE_AA, thickness=4)
            cv2.putText(img, str(lpText[:-2]), (x + 20, y - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 20, 0), lineType=cv2.LINE_AA, thickness=2)
            
            # Salvando tanto a imagem final, quanto a placamidentificada (para checar o que está errado, caso esteja)
            filename = f"./resultado/total{conta}.jpg"
            cv2.imwrite(filename, img)
            filename = f"./resultado/placa{conta}.jpg"
            conta += 1
            cv2.imwrite(filename, licensePlate)

## Referências
- https://analyticsindiamag.com/detecting-orientation-of-objects-in-image-using-pca-and-opencv/

- https://www.pyimagesearch.com/2020/09/21/opencv-automatic-license-number-plate-recognition-anpr-with-python/